In [ ]:
# ============================================================
# 003_extract_features.ipynb
# Step 3: Extract CNN features from cropped images
# 第3步：从裁剪后的图片中提取CNN特征向量
# ============================================================

import os
import pandas as pd
from pathlib import Path

# === 路径配置 Path Config ===
DATA_DIR  = r"D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data"
CSV_CLEAN = os.path.join(DATA_DIR, "images_clean.csv")      # 第00步输出的CSV
IMG_ROOT  = os.path.join(DATA_DIR, "img")                   # 原图目录
CROP_DIR  = os.path.join(DATA_DIR, "cropped_images")        # 第02步输出的裁剪图目录

# === 输出文件 Output Paths ===
OUT_FEAT  = os.path.join(DATA_DIR, "features.npy")          # 特征文件
OUT_KEYS  = os.path.join(DATA_DIR, "keys.csv")              # 图片对应信息

# === 模型参数 Model Parameters ===
MODEL_NAME   = "resnet18"           # 'resnet50' or 'mobilenet_v3_large'
IMAGE_SIZE   = 224
BATCH_GPU    = 64
BATCH_CPU    = 16
NUM_WORKERS  = 0                    # Windows 建议设为 0；若稳定可调为 2/4
SKIP_TRAIN   = True                 # True：仅提取 query + gallery
ITEM_DEDUP   = False                # True：每类仅保留一张图
DRY_RUN_N    = None                 # 设为 500 可快速测试流程；None = 全量

print(f"[INFO] DATA_DIR: {DATA_DIR}")
print(f"[INFO] CSV_CLEAN: {CSV_CLEAN}")
print(f"[INFO] CROP_DIR: {CROP_DIR}")


[INFO] DATA_DIR: D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data
[INFO] CSV_CLEAN: D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data\images_clean.csv
[INFO] CROP_DIR: D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data\cropped_images


In [2]:
# ============================================================
# Load cleaned CSV and point to cropped image paths
# 读取清洗后的CSV，并定位到裁剪图像
# ============================================================

df = pd.read_csv(CSV_CLEAN)

# 拼接绝对路径 Absolute path from relative path
def to_abs_from_root(rel):
    p = str(rel).replace("\\", "/")
    return os.path.join(IMG_ROOT, p)

df["image_abs"] = df["image_path"].apply(to_abs_from_root)

# 将原图路径映射为裁剪图路径
rel_paths = df["image_abs"].apply(lambda p: os.path.relpath(p, IMG_ROOT))
df["cropped_abs"] = rel_paths.apply(lambda r: os.path.join(CROP_DIR, r))

# 仅保留存在的图像
df = df[df["cropped_abs"].apply(os.path.exists)].reset_index(drop=True)
print(f"✅ Total usable images: {len(df)}")

# 若只提取 query + gallery
if SKIP_TRAIN:
    df = df[df["eval_split"].isin(["query", "gallery"])].reset_index(drop=True)
    print("After filtering query/gallery:", len(df), df["eval_split"].value_counts().to_dict())

# 若每个item仅保留一张
if ITEM_DEDUP:
    df = df.sort_values("image_path").drop_duplicates(["item_id","eval_split"]).reset_index(drop=True)
    print("After dedup:", len(df))

# 快速测试模式
if DRY_RUN_N:
    df = df.sample(min(DRY_RUN_N, len(df)), random_state=42).reset_index(drop=True)
    print("Dry run:", len(df))

df.head(3)


✅ Total usable images: 52712
After filtering query/gallery: 26830 {'query': 14218, 'gallery': 12612}


,image_path,eval_split,item_id_eval,item_id,x1,y1,x2,y2,image_abs,cropped_abs
0,WOMEN/Blouses_Shirts/id_00000001/02_1_front.jpg,gallery,id_00000001,id_00000001,50,49,208,235,D:\Icey\tcd\notebooks\dMining\Group Project\wo...,D:\Icey\tcd\notebooks\dMining\Group Project\wo...
1,WOMEN/Blouses_Shirts/id_00000001/02_2_side.jpg,query,id_00000001,id_00000001,119,48,136,234,D:\Icey\tcd\notebooks\dMining\Group Project\wo...,D:\Icey\tcd\notebooks\dMining\Group Project\wo...
2,WOMEN/Blouses_Shirts/id_00000001/02_3_back.jpg,gallery,id_00000001,id_00000001,50,42,213,240,D:\Icey\tcd\notebooks\dMining\Group Project\wo...,D:\Icey\tcd\notebooks\dMining\Group Project\wo...


In [3]:
# ============================================================
# Load pretrained CNN (ResNet50 / MobileNet)
# 载入预训练模型
# ============================================================

import torch
import torchvision
import torchvision.transforms as T

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Running on device: {device}")

if MODEL_NAME == "resnet50":
    model = torchvision.models.resnet50(
        weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2
    )
    model.fc = torch.nn.Identity()  # 移除最后分类层
    FEAT_DIM = 2048

elif MODEL_NAME == "resnet18":
    model = torchvision.models.resnet18(
        weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1
    )
    model.fc = torch.nn.Identity()  # 同样移除分类层，取倒数第二层特征
    FEAT_DIM = 512   # ResNet18 的特征维度是 512

elif MODEL_NAME == "mobilenet_v3_large":
    model = torchvision.models.mobilenet_v3_large(
        weights=torchvision.models.MobileNet_V3_Large_Weights.IMAGENET1K_V2
    )
    model.classifier = torch.nn.Identity()
    FEAT_DIM = 960

else:
    raise ValueError(f"Unsupported MODEL_NAME: {MODEL_NAME}")


model.eval().to(device)

transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])


🖥️ Running on device: cpu


In [4]:
# ============================================================
# Define dataset and dataloader
# 定义数据加载器
# ============================================================

from PIL import Image
from torch.utils.data import Dataset, DataLoader

class CroppedDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        r = self.df.iloc[idx]
        img = Image.open(r["cropped_abs"]).convert("RGB")
        x = self.transform(img)
        return x, r["image_path"], r["item_id"], r["eval_split"]

batch_size = BATCH_GPU if device == "cuda" else BATCH_CPU
dl = DataLoader(
    CroppedDataset(df, transform),
    batch_size=batch_size, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=(device=="cuda")
)
print(f"✅ Dataloader ready: {len(dl)} batches")


✅ Dataloader ready: 1677 batches


In [5]:
# ============================================================
# Forward pass and save features
# 批量提取特征并保存结果
# ============================================================

from tqdm import tqdm
import numpy as np
torch.set_grad_enabled(False)
if device == "cuda":
    torch.backends.cudnn.benchmark = True

feats, keys = [], []
desc = f"Extract({MODEL_NAME},{IMAGE_SIZE})"

for xb, img_paths, item_ids, splits in tqdm(dl, total=len(dl), desc=desc):
    xb = xb.to(device, non_blocking=True)
    with torch.cuda.amp.autocast(enabled=(device == 'cuda')):
        fb = model(xb)
        if fb.ndim == 4:  # 若输出维度为 [B,C,1,1]
            fb = torch.flatten(fb, 1)
        fb = fb.detach().cpu().numpy()
    # L2 normalization
    fb = fb / (np.linalg.norm(fb, axis=1, keepdims=True) + 1e-9)
    feats.append(fb)
    keys.extend(zip(img_paths, item_ids, splits))

feats = np.vstack(feats).astype("float32")
keys_df = pd.DataFrame(keys, columns=["image_path","item_id","eval_split"])

# === Save ===
np.save(OUT_FEAT, feats)
keys_df.to_csv(OUT_KEYS, index=False)
print(f"✅ Saved features to {OUT_FEAT} (shape={feats.shape})")
print(f"✅ Saved keys to {OUT_KEYS} (rows={len(keys_df)})")


Extract(resnet50,224):   0%|          | 0/1677 [00:00<?, ?it/s]C:\Users\Icey\AppData\Local\Temp\ipykernel_6716\4006215002.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device == 'cuda')):
Extract(resnet50,224): 100%|██████████| 1677/1677 [44:22<00:00,  1.59s/it]


✅ Saved features to D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data\features.npy (shape=(26830, 2048))
✅ Saved keys to D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data\keys.csv (rows=26830)
